# Исследование надежности заемщиков.
**Цель исследования** — нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**Ход исследования**. 
Входные данные от банка — статистика о платёжеспособности клиентов из файла `data.csv `.

Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы исследования. Вывод.

### Шаг 1. Обзор данных

Основной инструмент аналитика — pandas. Импортируем эту библиотеку.

In [1]:
import pandas as pd

Прочитаем файл `data.csv` из папки `/datasets` и сохраним его в переменной `data`:

In [2]:
data = pd.read_csv('/datasets/data.csv')

Выведем на экран первые 10-20 строк таблицы:

In [3]:
data.head(20) # получение первых 20 строк таблицы, чтобы увидеть пропущенные значение, например в 12 строке

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Тип данных: 5 — `object`, 5 столбцов целочисленные значения тип - `int64`, 2 столбца вещественные - float64.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок соблюден стиль.

Количество значений в столбцах различается. Это относится к столбцам `days_employed ` и  `total_income`. Значит, в данных есть пропущенные значения. При чём прослеживается **закономерность**: если нет данных об общем трудовом стаже, то и нет данных о ежемесячном доходе. С типами данных все в порядке, за исключением неудобного формата для `total_income`, округлим и переведем в int. Значения `days_employed ` ещё и часто отрицательные, *предварительно, эти данные для анализа нам не нужны*. Также есть проблемы с регистром в `education`.

### Шаг 2.1 Заполнение пропусков

In [5]:
data[data['days_employed'].isna()].head(10) # метод isna выявим пропуски. закономерность прослеживается

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [6]:
data.isna().sum() # посчитаем пропущенные значения 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
'Доля пропущенных значений в каждом из столбцов с пропусками: {:.0%}'.format(len(data[data['days_employed'].isna()])/data.shape[0])  

'Доля пропущенных значений в каждом из столбцов с пропусками: 10%'

Выяснили, что доля пропущенных значений в столбцах составляет 10%. Это большой показатель. Возможная причина наличия пропусков - нежелание заполнять информацию о своих ежемесячных доходах и трудовом стаже в анкетировании для банка. Или у людей нет подтвержденного дохода и стажа (работа в черную и тд). Заменим пропуски медианными значениями, которые лучше показывают ситуацию в выборке, если есть оторванные от основной массы лидеры.

In [8]:
income_grouped_median = data.groupby('income_type')['total_income'].median()   #посчитаем медиану дохода по типу занятости
display(income_grouped_median) #значения более или менее логичные

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [9]:
def bygroup(data, row): # функция заменяет NaN в row на медианное значение этого столбца у соответствующего income_type
    unique_inc_type = data['income_type'].unique()
    for type in unique_inc_type:
        data.loc[data['income_type'] == type, row] = data.loc[data['income_type'] == type, row].fillna(data[data['income_type'] == type]['total_income'].median())
    return data

In [10]:
data = bygroup(data, 'total_income')
# снова вызываем метод и подтверждаем замену пропусков

In [11]:
data.info() # проверяем 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [12]:
data.head(15) # проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [13]:
# столбец с днями трудового стажа сожержит странную информацию 
# даже при переводе ее в дни, получается, что трудовой стаж может быть больше возраста
# значения отрицательные
# для наших целей данный столбец не нужен, а значит его можно удалить
#del data['days_employed']

`days_employed` присутствуют слишком большие значения (возможно это стаж в часах, а не днях) и много отрицательных (возможно минус это тире) - оба вида недопустимы. **Скорее всего нужно запросить у источника дополнительную информацию (возможно ошибка при выгрузке), после внести исправления и проанализировать закономерности в некорректных значениях.** 

Но на данный момент преобразуем, чтобы не отвлекал, так как при выполнении поставленной задачи, эта информация не пригодится. 

In [14]:
data['days_employed'] = data['days_employed'].apply(abs) # избавляемся от отрицательных значений

Для замены пропусков медианными значениями используем transform.

In [15]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('dob_years')['days_employed'].transform("median"))

In [16]:
data.info() # проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [17]:
data['dob_years'] = data['dob_years'].astype('int') #приведём к целому значению


In [18]:
data['children'] = data['children'].abs() # выяснилось у кого-то отрицательное количество детей, скорее всего просто ошибка в знаке


In [19]:
data['education'] = data['education'].str.lower() # исправляем проблемы с регистром
data['family_status'] = data['family_status'].str.lower()
data['purpose'] = data['purpose'].str.lower()

In [20]:
data.head() #проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.3. Изменение типов данных.

In [21]:
data['total_income'] = data['total_income'].astype('int') # заменим тип total_income на int, т.к. точность до копеек не нужна

In [22]:
data.head() #проверяем

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

Посмотрим есть ли дубликаты в таблице методом duplicated - ищет одинаковые повторяющиеся значения

In [23]:
data.duplicated().sum() #общее количество дубликатов

71

In [24]:
data = data.drop_duplicates().reset_index(drop=True) #удаляем и восстанавливаем нумерацию
data.duplicated().sum() #проверяем

0

**Дополнительно поизучав данные на уровне подсчета уникальных значений (value_counts()) для каждого столбца, был выявлен еще ряд проблем:**
* `children` - содержит значение 20. От отрицательного мы избавились (возможно это была опечатка или отсутствие детей, но так как многие находятся\находились в браке остановимся на первом варианте). 20 - скорее опечатка (случайно нажали 0, печатая 2)
* `gender` - содержит пол XNA (только в 1 ячейке). Большинство в данных женщины, поэтому заменим на F.
* `purpose` - содержит опечатку - ремонт жильЮ (пока не знаю важно ли это, но исправим).

In [25]:
# исправим опечатку в purpose
data.loc[data['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверяем
data.loc[data['purpose'] == 'ремонт жилью']['purpose'].count()

0

In [26]:
# исправим значение пола
#data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
# проверяем
#data.loc[data['gender'] == 'XNA']['gender'].count()

In [27]:
# исправим значения 20 для возраста согласно предположению
data.loc[data['children'] == 20, 'children'] = 2
# проверяем
data.loc[data['children'] == 20]['children'].count()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [28]:
education_dict = data[['education', 'education_id']] # выделим словарь для education
education_dict = education_dict.drop_duplicates().reset_index(drop=True) # удалим дубликаты из словаря
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [29]:
family_status_dict = data[['family_status', 'family_status_id']] # выделим словарь для family_status
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) # удалим дубликаты из словаря
family_status_dict.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [30]:
data = data.drop(data.columns[[2, 4]], axis=1) #удаляем несколько столбцов из таблицы, используем функцию drop(), обращаясь к определенным столбцам, параметр axis=1 - двигаемся слева направо

In [31]:
data.head() #проверяем

,children,days_employed,education,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,высшее,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,среднее,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,среднее,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,среднее,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,среднее,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [32]:
data['total_income'].value_counts()

142594    1070
172357     502
118514     387
150447     145
144533       3
          ... 
179138       1
61724        1
99269        1
151371       1
126820       1
Name: total_income, Length: 18608, dtype: int64

In [33]:
def make_income_category(total_income): # для этого напишем функцию 
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if 1000001 <= total_income:
        return 'A'


data['total_income_category'] = data['total_income'].apply(make_income_category) #применим метод apply() для создания столбца с разбивкой по категориям

# проверяем
data['total_income_category'].value_counts()

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

In [34]:
data.head() # проверяем

,children,days_employed,education,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,высшее,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,среднее,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,среднее,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,среднее,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,среднее,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [35]:
data['purpose'].value_counts() #посмотрим на категории

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилья                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

На основании имеющихся данных можно выделить 4 основные категории. Вероятно данные вносились разными людьми, поэтому записаны по разному, но суть остается прежней.
Сформируем новый столбец `purpose_category`, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.


In [36]:
def purpose_category(list): #напишем функцию для разделения по категориям 
    if 'авто' in list:
        return "операции с автомобилем"
    if "образовани" in list:
        return "получение образования"
    if "свадьб" in list:
        return "проведение свадьбы"
    if "недвижим" in list:
        return "операции с недвижимостью"
    if "жиль" in list:
        return "операции с недвижимостью"

data['purpose_category'] = data['purpose'].apply(purpose_category)
data['purpose_category'].value_counts() #проверяем

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

In [37]:
data.head(10) #проверяем

,children,days_employed,education,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,высшее,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,среднее,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,среднее,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,среднее,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,среднее,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,высшее,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,высшее,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,среднее,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,высшее,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,среднее,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Чтобы ответить на этот и следующие вопросы, составим сводную таблицу методом pivot_table. Сведем все необходимые столбцы. Посмотрим есть ли зависимости.

In [38]:
data_pivot = data.pivot_table(index=['family_status_id', 'total_income_category'], columns='children', values='debt', aggfunc='count')
display(data_pivot)

children                                     0       1       2      3     4  \
family_status_id total_income_category                                        
0                A                         7.0     3.0     3.0    1.0   NaN   
                 B                      1749.0   737.0   399.0   68.0   4.0   
                 C                      5556.0  2215.0  1152.0  177.0  25.0   
                 D                       145.0    46.0    28.0    3.0   NaN   
                 E                        11.0     3.0     NaN    NaN   NaN   
1                A                         3.0     3.0     NaN    NaN   NaN   
                 B                       632.0   241.0    86.0   15.0   2.0   
                 C                      2048.0   742.0   264.0   39.0   6.0   
                 D                        43.0    13.0     4.0    2.0   NaN   
                 E                         4.0     1.0     1.0    NaN   NaN   
2                B                       130.0    19.0     4.0    3.0   NaN   
                 C                       698.0    58.0    20.0    3.0   1.0   
                 D                        19.0     4.0     NaN    NaN   NaN   
3                A                         1.0     2.0     NaN    NaN   NaN   
                 B                       188.0    73.0    19.0    1.0   1.0   
                 C                       583.0   238.0    62.0   10.0   NaN   
                 D                        12.0     3.0     2.0    NaN   NaN   
4                A                         1.0     1.0     NaN    NaN   NaN   
                 B                       547.0   105.0    14.0    1.0   NaN   
                 C                      1688.0   347.0    70.0    7.0   2.0   
                 D                        24.0     1.0     NaN    NaN   NaN   
                 E                         2.0     NaN     NaN    NaN   NaN   

children                                  5  
family_status_id total_income_category       
0                A                      NaN  
                 B                      4.0  
                 C                      2.0  
                 D                      1.0  
                 E                      NaN  
1                A                      NaN  
                 B                      NaN  
                 C                      2.0  
                 D                      NaN  
                 E                      NaN  
2                B                      NaN  
                 C                      NaN  
                 D                      NaN  
3                A                      NaN  
                 B                      NaN  
                 C                      NaN  
                 D                      NaN  
4                A                      NaN  
                 B                      NaN  
                 C                      NaN  
                 D                      NaN  
                 E                      NaN

Вызовем метод agg(), указывающий, какие именно функции применить к столбцу `debt`. Название столбца и сами функции запишем в словарь. И группируем по `children`.

In [39]:
data_grouped = data.groupby('children').agg({'debt': ['count', 'sum']})
display(data_grouped)

debt      
          count   sum
children             
0         14091  1063
1          4855   445
2          2128   202
3           330    27
4            41     4
5             9     0

In [40]:
data_grouped['conversion'] = (data_grouped['debt']['sum'] / data_grouped['debt']['count'])*100
display(data_grouped['conversion']) #посчитаем конверсию невозврата 

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: conversion, dtype: float64

**Доля невыплат меньше всего у тех у кого нет детей - 7,5%. А больше всего доля просрочки с 1, 2 и 4  детьми.  В целом небольшая разница между людьми с одним ребенком и несколькими, вторые возвращают вовремя чуть реже.**

##### Вопрос 2:

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [41]:
data_family_id = data.groupby(['family_status_id']).agg({'debt':['count', 'sum']})
display(data_family_id) # делаем тоже самое только по семейному статусу, где 
# 0 - женат\замужем 
# 1 - гражданский брак
# 2 - вдовец\вдова
# 3 - в разводе
# 4 - не женат\не замужем

debt     
                  count  sum
family_status_id            
0                 12339  931
1                  4151  388
2                   959   63
3                  1195   85
4                  2810  274

In [42]:
data_family_id[('debt', 'conv,%')] = data_family_id[('debt', 'sum')] / data_family_id[('debt', 'count')]*100
display(data_family_id) #посчитаем конверсию невозврата

debt               
                  count  sum    conv,%
family_status_id                      
0                 12339  931  7.545182
1                  4151  388  9.347145
2                   959   63  6.569343
3                  1195   85  7.112971
4                  2810  274  9.750890

**У людей в гражданском браке и не женатых получился примерно одинаковый процент невозврата по кредиту. Меньше всего просрочек у вдовцов\вдов.**

##### Вопрос 3:

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

На основании диапазонов `total_income_category` с категориями, проделаем такие же манипуляции что и ранее
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [43]:
data_total_income_category = data.groupby(['total_income_category']).agg({'debt':['count', 'sum']})
data_total_income_category[('debt', 'conv,%')] = data_total_income_category[('debt', 'sum')] / data_total_income_category[('debt', 'count')]*100
display(data_total_income_category)

debt                
                       count   sum    conv,%
total_income_category                       
A                         25     2  8.000000
B                       5042   356  7.060690
C                      16015  1360  8.492039
D                        350    21  6.000000
E                         22     2  9.090909

**Невозврат по кредиту самый высокий у людей с доходом менее 30тыс. У людей с высоким доходом меньше просрочек. С доходом от 30 до 200 тыс. много кредитных обязательств.**


##### Вопрос 4:

**Как разные цели кредита влияют на его возврат в срок?**

In [44]:
data_purpose_category = data.groupby(['purpose_category']).agg({'debt':['count', 'sum']})
data_purpose_category[('debt', 'conv,%')] = data_purpose_category[('debt', 'sum')] / data_purpose_category[('debt', 'count')]*100
display(data_purpose_category)

debt               
                          count  sum    conv,%
purpose_category                              
операции с автомобилем     4306  403  9.359034
операции с недвижимостью  10811  782  7.233373
получение образования      4013  370  9.220035
проведение свадьбы         2324  186  8.003442

**Больше всего просроченых кредитов в категории "операции с недвижимостью", но зато доля просрочек по категории 7,2% - самая низкая.
Категории "операции с автомобилем" и "получение образования" находятся посередине по количеству просрочек, а доля невыплат здесь значительно выше - около 9,3%.
И меньше всего просрочек соответственно в более мелкой категории "проведение свадьбы" - 8,0%.**

## Общий вывод:

В ходе анализа было удалено около 1% некорректных данных. В двух столбцах поменяли типы - для лучшей наглядности и более быстрой обработки, удалили дубликаты, сделали категоризацию, ипользовали функцию и ответили на все 4 поставленных вопроса. 

**Очень ярких закономерностей в просрочках по кредитам нет, но незначительные закономерности прослеживаются.**

Если описывать аватар клиента, который с наибольшей вероятностью допустит невыполнение кредитных обязательств, то он может выглядить так:
* Человек состоит в гражданском или зарегистрированном браке
* Его доход около 30тыс. рублей
* Скорее всего в семье двое и больше детей
* Кредит взят для улучшения жилищных условий или на покупку автомобиля (что необходимо при наличии большого семейства)